# Scenario Examples

This notebook demonstrates how to use the `Scenario` object to retrieve scenario inputs from the ETM API.

Make sure you have a valid `ETM_API_TOKEN` set in your environment.

## Structure

This notebook is organized into two main sections:
1. **Setup & Initialization** - Run these cells first to set up your environment and load a scenario
2. **Exploration Examples** - After setup is complete, these cells can be run in any order to explore different aspects of scenario data

#TODO: Expand of course

## Setup & Initialization

**Run these cells first!** The following cells set up your environment and load a scenario. Complete this section before exploring the examples below.

In [ ]:
# Environment Setup
# This cell configures the Python environment, imports required libraries, and sets up paths
# It also configures the notebook to hide detailed error tracebacks for cleaner output

import os, sys
import pprint
from pathlib import Path

# Set paths so we can import pyetm from the src directory
project_root = Path(os.getcwd()).parent
src_path     = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))  # so `import pyetm` works

# Configure cleaner error display for better notebook readability
ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                   exception_only=False, running_compiled_code=False):
    etype, value, tb = sys.exc_info()
    return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

print("✓ Environment setup complete")

In [ ]:
# Import pyETM and verify API connection
# This cell imports the necessary pyETM modules and checks that your API token is configured correctly

from pyetm.config.settings import get_settings
from pyetm.models import Scenario

# Verify API configuration
print("Using ETM API at    ",       get_settings().base_url)
print("Token loaded?       ",       bool(get_settings().etm_api_token))

if not get_settings().etm_api_token:
    print("⚠️  Warning: No ETM_API_TOKEN found. Please set your token in the environment.")
else:
    print("✓ API connection ready")

In [ ]:
# Load a scenario from the ETM API
# This cell connects to a specific scenario using its ID and loads all its data
# The scenario object will contain inputs, outputs, custom curves, and other configuration data

# Connect to your scenario by supplying the ID
scenario = Scenario.load(2690288)

print(f"✓ Scenario {scenario.id} loaded successfully")
print(f"  Title: {getattr(scenario, 'title', 'N/A')}")
print(f"  Total inputs: {len(scenario.inputs)}")
print(f"  User-modified inputs: {len(scenario.user_values())}")
print("\n🎉 Setup complete! You can now run any of the exploration examples below in any order.")

## Exploration Examples

**The cells below can be run in any order** after completing the setup section above. Each cell demonstrates different ways to explore and analyze scenario data using the pyETM API.

### Exploring User-Modified Values

The `user_values()` method returns a dictionary of all inputs that have been modified from their default values. This is useful for understanding what customizations have been made to a scenario.

In [ ]:
# Display all user-modified input values
# This shows only the inputs that have been changed from their default values
# The format is {input_key: user_value}

user_modifications = scenario.user_values()
print(f"Found {len(user_modifications)} user-modified inputs:")
print("\n" + "="*50)
pprint.pp(user_modifications)

### Analyzing Input Properties

Each input in a scenario has various properties that define its behavior:
- `key`: Unique identifier for the input
- `unit`: The unit of measurement (e.g., 'MW', '%', 'PJ')
- `disabled`: Whether the input is currently disabled
- `user`: The value set by the user (if any)
- `default`: The default value for this input

Additional properties for specific input types:
- **Float inputs**: `min` and `max` values defining valid ranges
- **Enumerable inputs**: `permitted_values` showing available options

In [ ]:
# Analyze units used across all inputs
# This gives you an overview of what types of measurements are used in the scenario
# Units like 'MW' (megawatts), '%' (percentage), 'PJ' (petajoules) indicate different input categories

units_used = set([input.unit for input in scenario.inputs])
print(f"Found {len(units_used)} different units used across {len(scenario.inputs)} inputs:")
print("\n" + "="*50)
pprint.pp(sorted(units_used))

In [ ]:
# Identify disabled inputs
# Disabled inputs are those that cannot be modified, often due to dependencies or constraints
# This helps understand which parameters are locked in the current scenario configuration

disabled_inputs = [input.key for input in scenario.inputs if input.disabled]
print(f"Found {len(disabled_inputs)} disabled inputs:")
print("\n" + "="*50)
pprint.pp(disabled_inputs)

In [ ]:
# Display default values for all inputs
# Default values represent the baseline scenario before any user modifications
# This is useful for understanding the reference case and for resetting modifications

default_values = { input.key: input.default for input in scenario.inputs }
print(f"Default values for {len(default_values)} inputs:")
print("\n" + "="*50)
pprint.pp(default_values)

In [ ]:
# Explore Float input constraints
# Float inputs have minimum and maximum values that define valid ranges
# This information is crucial for understanding input limitations and validation rules

float_inputs = [input for input in scenario.inputs if hasattr(input, "min") and hasattr(input, "max")]
print(f"Found {len(float_inputs)} float inputs with min/max constraints:")
print("\n" + "="*50)

float_constraints = [
    {
        "key":     input.key,
        "min":     input.min,
        "max":     input.max,
        "default": input.default,
        "unit":    input.unit
    }
    for input in float_inputs
]
pprint.pp(float_constraints)

### Working with Custom Curves

Custom curves represent time-series data that can be attached to specific inputs. These are typically used for:
- Load profiles (electricity demand over time)
- Production profiles (renewable energy output patterns)
- Availability curves (when technologies are available)
- Price curves (energy prices over time)

The curves contain hourly data points for an entire year (8760 hours).

In [ ]:
# List all custom curves attached to this scenario
# Custom curves are time-series data (usually hourly for a full year)
# They're used to define dynamic behavior like load profiles, production patterns, etc.

attached_curves = scenario.custom_curves.attached_keys()
print(f"Found {len(attached_curves)} custom curves attached to this scenario:")
print("\n" + "="*50)
pprint.pp(attached_curves)

In [ ]:
# Example: Display data from a specific curve
# This shows how to access the actual time-series data from a custom curve
# The curve_series() method returns the hourly values for the specified curve

curve_key = 'interconnector_1_import_availability'
if curve_key in scenario.custom_curves.attached_keys():
    curve_data = scenario.curve_series(curve_key)
    print(f"Curve data for '{curve_key}':")
    print(f"  Data points: {len(curve_data)}")
    print(f"  First 10 values: {curve_data[:10]}")
    print(f"  Data type: {type(curve_data)}")
else:
    print(f"Curve '{curve_key}' not found in this scenario.")
    print("Available curves:", list(scenario.custom_curves.attached_keys()))

### Exploring Sortables

Sortables represent ordered lists of technologies or components within the energy system. They define priority orders for:
- Merit order (which power plants run first)
- Heat source priorities
- Technology preferences

The order of items in sortables affects how the energy system model calculates results.

In [ ]:
# Display sortable configurations
# Sortables define the order/priority of different technologies in the energy system
# For example, merit order determines which power plants are dispatched first
# The order directly affects energy system calculations and results

sortables_data = scenario.sortables.as_dict()
print(f"Found {len(sortables_data)} sortable configurations:")
print("\n" + "="*50)
pprint.pp(sortables_data)

### Complete Scenario Data Export

The `model_dump()` method provides a complete export of all scenario data in a structured format. This includes:
- All input values (both default and user-modified)
- Custom curves data
- Sortables configuration
- Metadata about the scenario

This is useful for:
- Backup and restoration of scenario configurations
- Detailed analysis of scenario structure
- Integration with other tools or scripts

In [ ]:
# Export complete scenario data
# model_dump() returns a comprehensive dictionary containing all scenario data
# This includes inputs, curves, sortables, and metadata in a structured format
# Useful for backup, detailed analysis, or integration with other tools

complete_data = scenario.model_dump()
print(f"Complete scenario data structure:")
print(f"  Main keys: {list(complete_data.keys())}")
print(f"  Total size: {len(str(complete_data))} characters")
print("\n" + "="*50)
print("Note: Full data dump is very large. Showing structure only.")
print("Uncomment the line below to see the complete data:")
print("# pprint.pp(complete_data)")